### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [18]:
# Install spark
!pip install pyspark
!pip install findspark
!pip install pandas

In [105]:
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
findspark.init()
import pandas as pd
from pyspark.sql.functions import *
import tarfile
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [10]:
# Start session

sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at /tmp/ipykernel_416/1253009273.py:3 

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [98]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv'

--2022-05-13 23:07:18--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.2’

searchterms.csv.2   100%[===================>] 227.99K  --.-KB/s    in 0.001s  

2022-05-13 23:07:18 (150 MB/s) - ‘searchterms.csv.2’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe

In [26]:
df=spark.createDataFrame(pd.read_csv(url))
df.show()

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|        laptop|
| 12|   11|2021|        laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| gaming laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|        laptop|
+---+-----+----+--------------+
only showing top 20 rows



In [93]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)
row_count = df.count()
col_count = len(df.columns)
print(f'Total number of rows: {row_count}')
print(f'Total number of columns: {col_count}')


Total number of rows: 10000
Total number of columns: 4


In [28]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [91]:
df.printSchema()

root
 |-- day: long (nullable = true)
 |-- month: long (nullable = true)
 |-- year: long (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [32]:
df.select('searchterm').where(df.searchterm=='gaming laptop').count()


499

In [59]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)


In [110]:
df_count=df.groupBy("searchterm").count()
df_count.sort(desc("count")).show(5)

[Stage 53:======================================>              (147 + 11) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [99]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!wget -N "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
tf=tarfile.open
!tar -xvzf model.tar.gz

--2022-05-13 23:08:06--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘model.tar.gz’ not modified on server. Omitting download.

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_predic

In [103]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)
from pyspark.ml.regression import LinearRegressionModel



In [108]:
model = LinearRegressionModel.load('sales_prediction.model')

In [4]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg


In [109]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features', 'year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



22/05/13 23:11:25 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/05/13 23:11:25 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
